In [1]:
from edward.models import Normal
import tensorflow as tf

In [2]:
def alp_optimizer_apply_gradients(n, s_n, grads_and_vars):
    t = 0.1
    delta = 10e-3
    eta = 1e-1

    update_ops = []
    for i, (grad, var) in enumerate(grads_and_vars):
        update_s_n_op = s_n[i].assign( (t * grad**2) + (1 - t) * s_n[i] )
        update_ops.append(update_s_n_op)
        
        p_n_first = eta * n**(-.5 + delta)
        p_n_second = 1 / (1 + tf.sqrt(s_n[i]))
        p_n = p_n_first * p_n_second
        
        apply_grad_op = var.assign_add(-p_n * grad)
        update_ops.append(apply_grad_op)
    return update_ops

In [27]:
w1 = tf.Variable(tf.random_normal([]))
w2 = tf.Variable(tf.random_normal([]))
var_list = [w1, w2]

x = tf.constant([3., 4., 5.])
y = tf.constant([.8, .1, .1])

pred = tf.nn.softmax(x * w1 * w2)
loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred)))
grads = tf.gradients(loss, var_list)
grads_and_vars = list(zip(grads, var_list))

s_n = tf.Variable(tf.zeros(2))
n = tf.Variable(tf.constant(1.))

train = alp_optimizer_apply_gradients(n, s_n, grads_and_vars)
increment_n = n.assign_add(1.)

In [28]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

In [30]:
for _ in range(100):
    _, first_var, _, second_var = sess.run(train)
    loss_ = sess.run(loss)
    print(first_var, second_var, loss_)

1.21794 -0.528384 0.781685
1.23194 -0.554108 0.770444
1.24555 -0.578363 0.760364
1.25869 -0.601221 0.751355
1.27179 -0.622749 0.743265
1.2839 -0.643003 0.736142
1.29581 -0.662049 0.729792
1.30681 -0.679938 0.72423
1.31726 -0.696732 0.719333
1.32716 -0.712482 0.715035
1.3367 -0.727242 0.711253
1.34552 -0.741057 0.707969
1.35381 -0.753979 0.705108
1.3616 -0.766057 0.702621
1.36891 -0.777336 0.700463
1.37585 -0.787858 0.698589
1.38232 -0.797665 0.696972
1.38836 -0.806797 0.695578
1.39398 -0.815294 0.69438
1.39921 -0.823194 0.693351
1.40402 -0.830532 0.692471
1.40853 -0.837344 0.691716
1.41269 -0.843663 0.691072
1.41656 -0.849519 0.690522
1.42013 -0.854943 0.690054
1.42344 -0.859963 0.689655
1.42649 -0.864605 0.689316
1.42931 -0.868895 0.689029
1.43191 -0.872856 0.688785
1.43431 -0.876513 0.688579
1.43652 -0.879885 0.688405
1.43855 -0.882993 0.688257
1.44042 -0.885855 0.688133
1.44214 -0.88849 0.688029
1.44372 -0.890914 0.687941
1.44518 -0.893143 0.687866
1.44651 -0.895191 0.687804
1.44773